# Version Control and Git
<!-- requirement: pylib/__init__.py -->
<!-- requirement: pylib/draw_graph.py -->

Version controlling resources is a basic production concept. Broadly speaking it allows us to:

* Keep a history of our project
* Collaborate easily with others
* Work on different parts of the project simultaneously


We have probably all used version control in some context, whether it's keeping multiple versions of documents with different file names or a stack of papers with edits in red ink in the corner.  Code is especially suited to being version controlled because the source is just human readable characters (unlike something like a PDF which has pretty unreadable source). 

This notebook will introduce you to the basics of version control and illustrate some examples where it greatly helps the production process.  We will be using git as an example source control as it is quite prevalent in both industry and open source, but many other version control systems (such as Mercurial) have similar concepts.

We will be making extensive use of bash commands in this notebook, we will almost always be running `cd ~/vc_example` which will put us in a directory called `vc_example` (on the JupyterHub pods, this is an alias to `/home/jovyan/vc_example`.  We do that so that we have a clean environment with no other git repos in higher directories on the file tree.  This is just in case you decide to version your notebooks and miniprojects (Hint: This is probably a good idea! `#VersionControlEverything`).

## Versions


The most basic use of git is to keep track of versions of code.  git doesn't actually keep track of full versions per say, but instead a history of changes, or commits, that you have made to your source.  If we start from no code and simply keep track of all the changes we have made, we can easily reconstruct (or rather git can) the source at any stage of its development.  Each commit also comes with a message, which if properly written, can give us information into why a change was made and how it was done.  

To get started with git, we need to initialize a *git repository* by navigating to the directory we want to version control and running `git init`.  Lets create an empty directory, if its already there, we will remove it and create it again.

In [ ]:
if [ -d ~/vc_example ]; then
  rm -rf ~/vc_example
fi
mkdir ~/vc_example
cd ~/vc_example
git init

We also need to configure git when using it for the first time. At the very least we need to provide a user name and email.

In [ ]:
git config --global user.name "Jane Doe"
git config --global user.email "jane.doe@example.com"

Now we can create a file called `example.txt` and take a look at it.  We will put some text into the file by redirecting standard out into the file.  If you want to learn more about Unix redirects, check out the `PR_Bash` notebook!

In [ ]:
echo "I am an example" > example.txt
cat example.txt

Now we can add this file to staging.  This a way of telling git that you want to add some files to the next commit.  In other words you are telling git that when you make the next commit, you want it to keep track of the changes in the files that you add to the staging area.

You can also run `git add .` to add all files in the current directory (recursive), but this a dangerous move as there are often resources you don't want to add such as build artifacts.

In [ ]:
git add example.txt 

Now that we have added the file, we can make our first commit.  This uses the command `git commit`.  Without any flags this will open a default text editor to write a commit message.  Here we will pass the commit message directly in with the `-m` flag.  Make sure to write good commit messages, they are very useful!

Note, if you haven't set up git yet, you may be asked to set a username and an email; if so, follow the instructions!

In [ ]:
git commit -m "first commit"

Great, we have made our first commit!  Now lets add some more text to our example file. Here we ill use `>>` which will append text to the file instead of overwriting it.

In [ ]:
echo "more text" >> example.txt
cat example.txt 

We can use `git diff` to see the difference between our untracked files and the index (staging area) (and more generally between two states of a repository).

In [ ]:
git diff

There is a ton of information in the above output.  We can see exactly what lines in what files we changed and how we changed them. 

Now lets stage this file for commit.  Yes, you will need to do this every time you want to commit, even if the file has been staged in a previous commit.  After we will commit, lets look at the diff again.

In [ ]:
git add example.txt
git diff

There was no output, so what happened?  Since we have added the only file we have to the staging area, the difference between our current file system and the staging area is nothing!  Lets instead look at the difference between the current working tree (state of repo) and HEAD which in this case is the last commit.

In [ ]:
git diff HEAD

Now we see the difference!  Lets commit this new change with a reasonable commit message.

In [ ]:
git commit -m "added more text"

If we want to look at our commit history, we can use `git log`.  In this case this produces

In [ ]:
git log 

So this seems pretty simple, we make a series of changes to some files and keep track of the changes.  This nice for one person, but about more than one person?

## Branches


There is nothing that says we need to keep track of only one version of the history.  If we think of commits as a sequence of changes, at any point in time we can "branch" off of these changes and create different paths of changes.  In other words, I can keep two (or more) working copies of the code and so can anyone else!

Git branches are simply a specific history of commits, a single repository can have many branches which will often share part or most of the same commit history.  Branches are lightweight and cheap since they only carry information about what has changed, they don't need to save entire version of the source code.  

Lets first check what branch we are currently on, we can do this with `git branch`

In [ ]:
git branch

We are on the `master` branch.  Usually this is the branch that is considered the main branch, or production branch.  If we want to make a branch from `master` (or from any branch we are currently on), we can use `git checkout -b <branch_name`.  Lets make a branch called `dev`.

In [ ]:
git checkout -b dev

Now lets look at our branches

In [ ]:
git branch

Now lets take a look at our history, notice its the same as before!

In [ ]:
git log 

Now we are going to add a new file, commit it to `dev`.

In [ ]:
echo "I am a second file" > second_file.txt
git add second_file.txt
git commit -m "added second_file"

In [ ]:
git log 

Lets now look at the log for `dev`, switch back to `master` with `git checkout <branch_name>` and look at the log for `master`!  We are going to add some options to the log command to make viewing things a bit nicer.  Remember HEAD just points to the current committed state of the active branch.

In [ ]:
git log --all --decorate --oneline --graph --color
echo "############################"
echo "switching to master"
echo "############################"
git checkout master
git log --all --decorate --oneline --graph --color

So we only have the change on `dev`, but not on `master`!  What about the actual file system for `master`?

In [ ]:
ls

It doesn't even have `second_file.txt`.  There are two really important things this allows us to do:

1. Work on things with someone else.  Two (or more) people can keep version of the repository and make their own changes, they can even keep all these different version in the same repository as separate branches.
2. We can have a master branch which is stable and tracks production.  Side branches are where we add new functionality and make changes.  So if I have changed a few files and suddenly I need to fix a bug on `master`, I can just make a new branch from `master` and fix that bug, I don't need to undo my changes on another branch

Of course these advantages make an implicit assumption that we can somehow merge these two branches together.  This is our next task.

## Merges


Merging in git is when two branches are combined together.  One can think of this as the changes from both branches being applied to the point at which they diverged from each other.  For the most part, especially if branches are merged often in small chunks there aren't too many problems to this, but you might imagine there are cases where the two branches change the same exact thing.  We will deal with this more complicated case in a sec.  First lets merge our `dev` branch into `master`.  We can do this by using the `git merge <branch_name>` command from the master branch (which we are currently on).

In [ ]:
git merge dev

Now lets look at the history for `master`.

In [ ]:
git log --all --decorate --oneline --graph --color

At this point it is the same as `dev`.  Because the `dev` branch had only one extra commit, there was a "obvious" way to merge them together.  Git is actually pretty smart about how to merge things together and often you will be able to merge without any trouble.

Unfortunately, this is not always the case.  Lets make a commit to `master` where we change the file `second_file.txt`.

In [ ]:
echo "this is the second file and its different" > second_file.txt
git add second_file.txt
git commit -m "changed the second file"

Now lets switch to the `dev` branch and make a change to `second_file.txt`, a different change than the change we made one the master branch.  This situation might arise when two people are working on the same area of the code and make a different change to it on their own branches.

In [ ]:
git checkout dev
echo "this is the second file, dev version" > second_file.txt
git add second_file.txt
git commit -m "changed the second file in dev"

Now lets switch back to the master branch and try to merge `dev`.  We could also merge `master` into `dev` and would have the same problem, namely, how does git choose which change to keep?

In [ ]:
git checkout master
git merge dev

In this case since the same line of code was changed, git can't automatically decide which change we want, so it will throw a CONFLICT.  In other words, it can't choose the order in which to apply the changes, it needs our input.

Lets visualize what happened.  

In [ ]:
git log --all --decorate --oneline --graph --color

Lets take a look at the file in question.

In [ ]:
cat second_file.txt

git has added some information to the file regarding the conflicts.  Many modern text editors (and some not so modern ones) will have capabilities to allow you to easily resolve these conflicts.  What you must do is choose one of the changes, in this case either from `HEAD` or `dev` and then make a commit with those changes. 

Here we will choose the `dev` version of the text and then commit it. Notice that git now records both of the commits in the log as well as the commit which resolved the merge conflict.  This makes sense, if we were to apply one change, then the other, then the merge conflict commit, we end up in the correct state.  Lets take a look at the log and then a visualization of the current git history

In [ ]:
echo "this is the second file, dev version" > second_file.txt
git add second_file.txt
git commit -m "resolve merge conflicts"
git log --all --decorate --oneline --graph --color

But after all this merging, what has happened to `dev`?  Lets take a look at the git log for the `dev` branch.

In [ ]:
git checkout dev
git log --all --decorate --oneline --graph --color

It still has the same history as before.  We could merge `master` into `dev`, but often its the case that a similar situation will arise when someone else on your team merges their branch into `master` while you are working on a change yourself.  Lets say we are adding a fancy new python program to say hello to us.  We will write it to file here and commit it to the `dev` branch.

In [ ]:
cat << EOF > hello_world.py
def hello_world():
    print "hello world"
if __name__ == '__main__':
    hello_world()
EOF
git add hello_world.py
git commit -m "added hello_world.py"

Lets look at our current git log and a visualization of the state of both `master` and `dev`.

In [ ]:
git log --all --decorate --oneline --graph --color

Now we are in a similar situation to before.  We could of course merge these branches together, but then our history would be a bit more complicated, we would need to keep track of a merge commit. Instead of merging, we can also rebase.

## Rebasing


If we rebase branch A on branch B, then all the commit in branch A will recreated as commits on top of the head of branch B.  This will create a nice history (by rewriting it) that does not have any forks.  In general you only want to rebase branches that other are not also working with (don't rebase master on another branch with any great frequency) or you will be left with many merge conflicts.

We can rebase with the `git rebase <branch_name>` command.  Of course we can also get conflicts here, we will see an example of that in a bit.  For now lets just rebase.

In [ ]:
git rebase master

Now we can look at the history, note that the commit hash has changed for commit `added hello_world.py`

In [ ]:
git log --all --decorate --oneline --graph --color

This history looks somewhat nicer than after the merge, in fact it has the same history as `master` with only one extra commit now.

We won't go into it here, but another use for rebasing is so called interactive rebase where you can combine commits and alter them as they get rebased using the `-i` flag.  This is often used to clean up the git history so that each commit is a nice unit of change, even when it didn't initially start that way. 

The downside to this strategy is that altering git history changes the natural story of the code.  Its always a trade off between sensibility and not changing history, you and your team will need to decide how to do this.

With regards to teams, so far we have dealt with only a single repository, but git offers us much more power in form of remotes.

## Remotes


Remotes allow us to have a local repository that tracks another repository (which does not have to live on the same computer).  This is the basis behind sites such as [GitHub](https://github.com) which manage repositories in a cloud environment.  Having this centralized repository means that there is a single source of truth and the local repositories are just copies of this repo, albeit not in general in the most recent state.  One nice thing about git in particular (this is not true of all version control software) is that it allows complete independence from the remote repository during development until we want to add our changes to the central source

To start we need to make a copy or `clone` a central repository.  We can do this with the `git clone <repo>` command.  For example if we wanted to clone the repository for `requests` which is an open source Python package for making web requests (also generally regarded as a model of proper Python development), we could simply run

`git clone git@github.com:requests/requests.git`

or

`git clone https://github.com/requests/requests.git`

depending on whether we were using ssh keys or username and password to authenticate ourselves.

In fact, lets do this!, we will clone it again to our home directory by changing into that.

In [ ]:
if [ -d ~/requests ]; then
  rm -rf ~/requests
fi
cd ~/
git clone https://github.com/requests/requests.git  

Lets take a look at this log

In [ ]:
cd ~/requests
git log --decorate --oneline --graph --color -n 80

Clearly there is some interesting stuff happening here!  Remember, even though its a remote, its still just a git repo, so everything we already learned still applies!  

Now lets check what the remote settings are for our local repo, we can do this with the `git remote -v` command.

In [ ]:
git remote -v

This tells us which git repositories are set for the `fetch` and `push` operations for remote `origin`.  You can have more than one remote. For example, when using Heroku, you will often have a remote called `heroku`.  If you have ever run `git push heroku master`, you are pushing to the master branch of the `heroku` remote.

Fetch and push are ways to sync the local and remote repository in opposite directions.  When one pushes, you are asking the remote repository to accept the changes made since the last time these repos were synced, and fetch (or pull) updates the local repo with changes from the remote repo.

## Common Git Workflows


There are several common workflows that utilize git.  We will go over two of them that the author has used, but do be aware, there are many ways of using version control and it will depend heavily on your company.

### Workplace


Here the workflow will operate on the feature branch model.  The master branch will track production (in principle the master branch should be able to be deployed at **any** time), and when one wants to make a change to the codebase, whether squashing bugs or adding features, one creates a branch based on `master` and codes there. 
When the bugs are squashed or the amazing new feature is ready to go, one opens up a pull request to merge the feature branch into the master branch.  Usually this is the point where continuous integration tests are run on the side branch.  The code is then reviewed and changes are made until all the reviewers are satisfied. 

At this point, the branch is merged into `master` (possibly also rebase off `master` and then merged, or squashed and then merged) and another set of continuous integration tests are run on `master`.  If these pass `master` is ready to be deployed and its time to start finding the next project.

### Open Source


Here the workflow will operate on the forked repository model.  If one wants to make a change to Scikit-learn, one forks the Scikit-learn repository and makes changes on any branch in the forked repository.  Once these changes are made, one opens a pull request from the forked repository into the master branch in the main repository.  The rest of the process proceeds as above.

Sources and Resources:

* [Merging vs Rebasing](https://www.atlassian.com/git/tutorials/merging-vs-rebasing)

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*